In [13]:
bbox = [79, 115, 467, 333]

In [28]:
from libcom import get_composite_image

comp, _ = get_composite_image('realistic/images/medium/n02924116_42328.jpg', 'output_mask.png', 'BG/n02924116_42328.JPEG', [79, 115, 467, 333], 'gaussian')

No module 'xformers'. Proceeding without it.


/opt/conda/envs/Libcom/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


In [20]:
from libcom import color_transfer

In [30]:
import cv2

cv2.imwrite('naive.jpg', comp)

True

In [21]:
from libcom import ImageHarmonizationModel

CDTNet = ImageHarmonizationModel(device=0, model_type='CDTNet')

In [24]:
from libcom import color_transfer

trans_img1 = color_transfer('naive.jpg', 'test_0.png')

cv2.imwrite('trans_naive.jpg', comp)

True

In [22]:
CDT_result1 = CDTNet('naive.jpg', 'test_0.png')

In [23]:
cv2.imwrite('naive_harmo.jpg', comp)

True

In [19]:
import torch
# get the original mask
mask_path = "FG/n02924116_42328_mask.png"
bw_mask = np.array(Image.open(mask_path).convert('L'))

binary_mask = (bw_mask > 128).astype(np.uint8)

mask_tensor = torch.from_numpy(binary_mask).float()
mask_tensor = mask_tensor.unsqueeze(0).unsqueeze(0)

In [20]:
print(mask_tensor)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])


In [29]:
from PIL import Image

img = Image.open('naive.jpg')

img = img.crop((0, 0, img.size[0], img.size[1] - 12))
         
img.save('naive_cropped.jpg')

In [23]:
bbox = [79, 115, 467, 333]

input_label = np.array([1])
center_x = (bbox[0] + bbox[2]) // 2  # Average of x1 and x2
center_y = (bbox[1] + bbox[3]) // 2  # Average of y1 and y2
center_point = np.array([[center_x, center_y]])

In [24]:
from PIL import Image
import numpy as np

device = "cuda"
from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry["vit_h"](checkpoint="./sam_vit_h_4b8939.pth")
sam.to(device)
predictor = SamPredictor(sam)
predictor.set_image(np.array(Image.open('naive.jpg')))
masks, scores, logits = predictor.predict(
    point_coords=center_point,
    point_labels=input_label,
    multimask_output=False
)

In [25]:
masks

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])

In [55]:
from libcom import get_composite_image
import json

def post_process(occ_level, image_path, predictor):
    
    # get annotation
    img_id = image_path.split('/')[-1].split('.')[0]
    
    # get occluder bbox from annotation
    anno_path = f'realistic/annotations/{occ_level}/{img_id}.json'
    comp_path = f'realistic/images/{occ_level}/{img_id}.jpg'
    
    bg_path = f'BG/{img_id}.JPEG'
    
    with open(anno_path) as f:
        d = json.load(f)
        
    bbox = d['occluder_box']
    
    # get the center of the occluder bbox
    center_x = (bbox[0] + bbox[2]) // 2  # Average of x1 and x2
    center_y = (bbox[1] + bbox[3]) // 2  # Average of y1 and y2
    center_point = np.array([[center_x, center_y]])
    
    input_label = np.array([1])
    
    predictor.set_image(np.array(Image.open(comp_path)))
    masks, scores, logits = predictor.predict(
        point_coords=center_point,
        box=np.array([bbox]),
        point_labels=input_label,
        multimask_output=False
    )
    
    mask_image = (masks[0].astype(np.uint8) * 255)

    # Save the mask as a PNG
    Image.fromarray(mask_image).save('temp_mask.png')
    
    comp, _ = get_composite_image(comp_path, 'temp_mask.png',  bg_path, bbox)
    
    cv2.imwrite(f'naive_{img_id}.jpg', comp)
    # mask the occluder out using the 

In [57]:
device = "cuda"
from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry["vit_h"](checkpoint="./sam_vit_h_4b8939.pth")
sam.to(device)
predictor = SamPredictor(sam)

with open('file_list.txt', 'r') as names:
    i = 0
    for name in names:
        occ_level, bg_img_path = name.strip().split(' ')
        
        post_process(occ_level, bg_img_path, predictor)
        i+=1
        if i == 5:
            break